# (2)

## a.

In [1]:
import pandas as pd
import numpy as np
import requests
import os 
import psycopg2
import zipfile
import io
from sqlalchemy import create_engine

In [2]:
postgres_pwd = os.environ['POSTGRES_PASSWORD']

## b.

In [3]:
#download the world bank data
url = 'https://databank.worldbank.org/data/download/ESG_csv.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [4]:
#download the V-dem data
url = 'https://v-dem.net/media/datasets/Country_Year_V-Dem_Core_CSV_v12.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

## c.

In [5]:
# First, let's focus on the V-Dem data ('Country_Year_V-Dem_Core_CSV_v12/V-Dem-CY-Core-v12.csv'). Use pandas methods to perform the following tasks:

v_dem_file = 'Country_Year_V-Dem_Core_CSV_v12/V-Dem-CY-Core-v12.csv'
v_dem_df = pd.read_csv(v_dem_file)
v_dem_df.head()

,country_name,country_text_id,country_id,year,historical_date,project,historical,histname,codingstart,codingend,...,v2xme_altinf_sd,v2xps_party,v2xps_party_codelow,v2xps_party_codehigh,v2x_divparctrl,v2x_feduni,v2xca_academ,v2xca_academ_codelow,v2xca_academ_codehigh,v2xca_academ_sd
0,Mexico,MEX,3,1789,1789-12-31,1,1,Viceroyalty of New Spain,1789,2021,...,0.671,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,Mexico,MEX,3,1790,1790-12-31,1,1,Viceroyalty of New Spain,1789,2021,...,0.671,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,Mexico,MEX,3,1791,1791-12-31,1,1,Viceroyalty of New Spain,1789,2021,...,0.671,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,Mexico,MEX,3,1792,1792-12-31,1,1,Viceroyalty of New Spain,1789,2021,...,0.671,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,Mexico,MEX,3,1793,1793-12-31,1,1,Viceroyalty of New Spain,1789,2021,...,0.671,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [6]:
#Keep only the 'country_text_id', 'country_name','year', 'v2x_polyarchy' columns.

v_dem_df = v_dem_df[['country_text_id', 'country_name','year', 'v2x_polyarchy']]
v_dem_df.head()

,country_text_id,country_name,year,v2x_polyarchy
0,MEX,Mexico,1789,0.026
1,MEX,Mexico,1790,0.026
2,MEX,Mexico,1791,0.026
3,MEX,Mexico,1792,0.026
4,MEX,Mexico,1793,0.026


In [7]:
#Use the .query() method to keep only the rows in which year is greater than or equal to 1960 and less than or equal to 2021
v_dem_df = v_dem_df.query("year >= 1960 and year <= 2021")
v_dem_df

,country_text_id,country_name,year,v2x_polyarchy
171,MEX,Mexico,1960,0.231
172,MEX,Mexico,1961,0.232
173,MEX,Mexico,1962,0.234
174,MEX,Mexico,1963,0.234
175,MEX,Mexico,1964,0.234
...,...,...,...,...
25976,ZZB,Zanzibar,2017,0.259
25977,ZZB,Zanzibar,2018,0.257
25978,ZZB,Zanzibar,2019,0.254
25979,ZZB,Zanzibar,2020,0.251


In [8]:
#Rename 'country_text_id' to 'country_code', 'country_name' to 'country_name_vdem', and 'v2x_polyarchy' to 'democracy'.

v_dem_df = v_dem_df.rename({'country_text_id': 'country_code',
                              'country_name':'country_name_vdem',
                              'v2x_polyarchy':'democracy'},
                            axis=1)
v_dem_df

,country_code,country_name_vdem,year,democracy
171,MEX,Mexico,1960,0.231
172,MEX,Mexico,1961,0.232
173,MEX,Mexico,1962,0.234
174,MEX,Mexico,1963,0.234
175,MEX,Mexico,1964,0.234
...,...,...,...,...
25976,ZZB,Zanzibar,2017,0.259
25977,ZZB,Zanzibar,2018,0.257
25978,ZZB,Zanzibar,2019,0.254
25979,ZZB,Zanzibar,2020,0.251


In [9]:
#Sort the rows by 'country_code' and 'year' in ascending order. [2 points]

v_dem_df.sort_values(by=['country_code', 'year'], ascending = [True, True])

,country_code,country_name_vdem,year,democracy
5404,AFG,Afghanistan,1960,0.081
5405,AFG,Afghanistan,1961,0.083
5406,AFG,Afghanistan,1962,0.083
5407,AFG,Afghanistan,1963,0.086
5408,AFG,Afghanistan,1964,0.098
...,...,...,...,...
25976,ZZB,Zanzibar,2017,0.259
25977,ZZB,Zanzibar,2018,0.257
25978,ZZB,Zanzibar,2019,0.254
25979,ZZB,Zanzibar,2020,0.251


## d.

In [10]:
#Next let's focus on the country-level World Bank data ('ESGCountry.csv'). Use pandas methods to perform the following tasks:

esg_country_df = pd.read_csv('ESGCountry.csv')
esg_country_df

,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,...,Government Accounting concept,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Unnamed: 30
0,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,The reporting period for national accounts dat...,South Asia,Low income,AF,...,Consolidated central government,Enhanced General Data Dissemination System (e-...,1979,"Demographic and Health Survey, 2015","Integrated household survey (IHS), 2016/17",NaN,NaN,NaN,2018.0,NaN
1,AGO,Angola,Angola,People's Republic of Angola,AO,Angolan kwanza,The World Bank systematically assesses the app...,Sub-Saharan Africa,Lower middle income,AO,...,Budgetary central government,Enhanced General Data Dissemination System (e-...,2014,"Demographic and Health Survey, 2015/16","Integrated household survey (IHS), 2008/09",NaN,NaN,NaN,2018.0,NaN
2,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,NaN,Europe & Central Asia,Upper middle income,AL,...,Consolidated central government,Enhanced General Data Dissemination System (e-...,2020 (expected),"Demographic and Health Survey, 2017/18",Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,NaN
3,AND,Andorra,Andorra,Principality of Andorra,AD,Euro,NaN,Europe & Central Asia,High income,AD,...,NaN,NaN,2011. Population figures compiled from adminis...,NaN,NaN,Yes,NaN,NaN,2018.0,NaN
4,ARB,Arab World,Arab World,Arab World,1A,NaN,Arab World aggregate. Arab World is composed o...,NaN,NaN,1A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,WSM,Samoa,Samoa,Samoa,WS,Samoan tala,The reporting period for national accounts dat...,East Asia & Pacific,Lower middle income,WS,...,Budgetary central government,Enhanced General Data Dissemination System (e-...,2020 (expected),"Multiple Indicator Cluster Survey, 2019/20","Expenditure survey/budget survey (ES/BS), 2013",NaN,2009,NaN,2018.0,NaN
235,YEM,Yemen,"Yemen, Rep.",Republic of Yemen,YE,Yemeni rial,The World Bank systematically assesses the app...,Middle East & North Africa,Low income,RY,...,NaN,Enhanced General Data Dissemination System (e-...,2004,"Demographic and Health Survey, 2013","Expenditure survey/budget survey (ES/BS), 2014",NaN,NaN,2012.0,2015.0,NaN
236,ZAF,South Africa,South Africa,Republic of South Africa,ZA,South African rand,Fiscal year end: March 31; reporting period fo...,Sub-Saharan Africa,Upper middle income,ZA,...,Consolidated central government,Special Data Dissemination Standard (SDDS),2011,"Demographic and Health Survey, 2016","Expenditure survey/budget survey (ES/BS), 2014/15",NaN,2007,2010.0,2018.0,NaN
237,ZMB,Zambia,Zambia,Republic of Zambia,ZM,New Zambian kwacha,National accounts data were rebased to reflect...,Sub-Saharan Africa,Low income,ZM,...,Budgetary central government,Enhanced General Data Dissemination System (e-...,2020 (expected),"Demographic and Health Survey, 2018","Integrated household survey (IHS), 2015",NaN,NaN,1994.0,2018.0,NaN


In [11]:
#Keep only the 'Country Code', 'Table Name', 'Long Name', 'Currency Unit', 'Region', and 'Income Group' columns.

esg_country_df = esg_country_df[['Country Code', 'Table Name', 'Long Name', 'Currency Unit', 'Region', 'Income Group']]
esg_country_df.head()

,Country Code,Table Name,Long Name,Currency Unit,Region,Income Group
0,AFG,Afghanistan,Islamic State of Afghanistan,Afghan afghani,South Asia,Low income
1,AGO,Angola,People's Republic of Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
2,ALB,Albania,Republic of Albania,Albanian lek,Europe & Central Asia,Upper middle income
3,AND,Andorra,Principality of Andorra,Euro,Europe & Central Asia,High income
4,ARB,Arab World,Arab World,NaN,NaN,NaN


In [12]:
#Rename 'Country Code' to 'country_code', 'Table Name' to 'country_name_wb', 'Long Name' to 'country_longname', 
#'Currency Unit' to 'currency_unit', 'Region' to 'region', and 'Income Group' to 'income_group'.

esg_country_df = esg_country_df.rename({'Country Code': 'country_code',
                                        'Table Name':'country_name_wb',
                                        'Long Name':'country_longname',
                                        'Currency Unit':'currency_unit',
                                        'Region':'region',
                                        'Income Group':'income_group'},
                                       axis=1)
esg_country_df.head()

,country_code,country_name_wb,country_longname,currency_unit,region,income_group
0,AFG,Afghanistan,Islamic State of Afghanistan,Afghan afghani,South Asia,Low income
1,AGO,Angola,People's Republic of Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
2,ALB,Albania,Republic of Albania,Albanian lek,Europe & Central Asia,Upper middle income
3,AND,Andorra,Principality of Andorra,Euro,Europe & Central Asia,High income
4,ARB,Arab World,Arab World,NaN,NaN,NaN


In [13]:
#The countries in this dataset include various grouping of countries such as "Europe & Central Asia". 
#We want to remove these observations from the data. Use the .query() method to remove the rows in which 'country_name_wb' is equal to one of these non-countries. 
#(Hint: within the .query() method, you can reference an external Python variable such as noncountries by placing an @ sign in front of the variable name.) 

noncountries = ["Arab World", "Central Europe and the Baltics","Caribbean small states",
                "East Asia & Pacific (excluding high income)","Early-demographic dividend","East Asia & Pacific",
                "Europe & Central Asia (excluding high income)","Europe & Central Asia", "Euro area",
                "European Union","Fragile and conflict affected situations","High income",
                "Heavily indebted poor countries (HIPC)","IBRD only","IDA & IBRD total",
                "IDA total","IDA blend","IDA only",
                "Latin America & Caribbean (excluding high income)","Latin America & Caribbean","Least developed countries: UN classification",
                "Low income","Lower middle income","Low & middle income",
                "Late-demographic dividend","Middle East & North Africa","Middle income",
                "Middle East & North Africa (excluding high income)","North America","OECD members",
                "Other small states","Pre-demographic dividend","Pacific island small states",
                "Post-demographic dividend","Sub-Saharan Africa (excluding high income)","Sub-Saharan Africa",
                "Small states","East Asia & Pacific (IDA & IBRD)","Europe & Central Asia (IDA & IBRD)",
                "Latin America & Caribbean (IDA & IBRD)","Middle East & North Africa (IDA & IBRD)","South Asia",
                "South Asia (IDA & IBRD)","Sub-Saharan Africa (IDA & IBRD)","Upper middle income", "World"]

esg_country_df = esg_country_df.query("country_name_wb != @noncountries")
esg_country_df

,country_code,country_name_wb,country_longname,currency_unit,region,income_group
0,AFG,Afghanistan,Islamic State of Afghanistan,Afghan afghani,South Asia,Low income
1,AGO,Angola,People's Republic of Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
2,ALB,Albania,Republic of Albania,Albanian lek,Europe & Central Asia,Upper middle income
3,AND,Andorra,Principality of Andorra,Euro,Europe & Central Asia,High income
5,ARE,United Arab Emirates,United Arab Emirates,U.A.E. dirham,Middle East & North Africa,High income
...,...,...,...,...,...,...
234,WSM,Samoa,Samoa,Samoan tala,East Asia & Pacific,Lower middle income
235,YEM,"Yemen, Rep.",Republic of Yemen,Yemeni rial,Middle East & North Africa,Low income
236,ZAF,South Africa,Republic of South Africa,South African rand,Sub-Saharan Africa,Upper middle income
237,ZMB,Zambia,Republic of Zambia,New Zambian kwacha,Sub-Saharan Africa,Low income


## e. 

In [14]:
wb_ts_df = pd.read_csv('ESGData.csv')
wb_ts_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2050,Unnamed: 67
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,86.232384,86.478597,86.722685,86.937933,87.040774,87.235539,87.307068,NaN,NaN,NaN
1,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,88.015356,88.681886,89.195062,90.324659,88.910749,89.999946,90.277735,NaN,NaN,NaN
2,Arab World,ARB,Adjusted savings: natural resources depletion ...,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,10.297891,6.237703,5.203895,6.480083,8.479070,7.437145,4.376374,NaN,NaN,NaN
3,Arab World,ARB,Adjusted savings: net forest depletion (% of GNI),NY.ADJ.DFOR.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,0.084948,0.099784,0.095056,0.095498,0.051058,0.062195,0.079081,NaN,NaN,NaN
4,Arab World,ARB,Agricultural land (% of land area),AG.LND.AGRI.ZS,NaN,30.981194,30.982443,31.006834,31.017781,31.042245,...,36.529508,36.565532,36.627752,36.673546,36.659112,36.597936,36.659647,NaN,NaN,NaN


In [15]:
#Keep only the columns named 'Country Code', 'Country Name', and 'Indicator Code', or begin with '19' or '20'. 
#(Don't type in all the years individually. Instead, use code that finds all columns that begin '19' or '20'.)
columns_1 = ['Country Code', 'Country Name', 'Indicator Code']
columns_2 = [x for x in wb_ts_df.columns if x.startswith('19') or x.startswith('20')]
wb_ts_df = wb_ts_df[columns_1 + columns_2]
wb_ts_df.head()

,Country Code,Country Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2050
0,ARB,Arab World,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85.932567,86.232384,86.478597,86.722685,86.937933,87.040774,87.235539,87.307068,NaN,NaN
1,ARB,Arab World,EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,88.992620,88.015356,88.681886,89.195062,90.324659,88.910749,89.999946,90.277735,NaN,NaN
2,ARB,Arab World,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.638397,10.297891,6.237703,5.203895,6.480083,8.479070,7.437145,4.376374,NaN,NaN
3,ARB,Arab World,NY.ADJ.DFOR.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.062625,0.084948,0.099784,0.095056,0.095498,0.051058,0.062195,0.079081,NaN,NaN
4,ARB,Arab World,AG.LND.AGRI.ZS,NaN,30.981194,30.982443,31.006834,31.017781,31.042245,31.05018,...,36.532755,36.529508,36.565532,36.627752,36.673546,36.659112,36.597936,36.659647,NaN,NaN


In [16]:
#Then use the .drop() method to delete the column named '2050'

wb_ts_df = wb_ts_df.drop(['2050'], axis = 1)
wb_ts_df.head()

,Country Code,Country Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ARB,Arab World,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85.644218,85.932567,86.232384,86.478597,86.722685,86.937933,87.040774,87.235539,87.307068,NaN
1,ARB,Arab World,EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,87.039588,88.992620,88.015356,88.681886,89.195062,90.324659,88.910749,89.999946,90.277735,NaN
2,ARB,Arab World,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.859225,11.638397,10.297891,6.237703,5.203895,6.480083,8.479070,7.437145,4.376374,NaN
3,ARB,Arab World,NY.ADJ.DFOR.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.032266,0.062625,0.084948,0.099784,0.095056,0.095498,0.051058,0.062195,0.079081,NaN
4,ARB,Arab World,AG.LND.AGRI.ZS,NaN,30.981194,30.982443,31.006834,31.017781,31.042245,31.05018,...,36.484107,36.532755,36.529508,36.565532,36.627752,36.673546,36.659112,36.597936,36.659647,NaN


In [17]:
#Rename 'Country Code' to'country_code', 'Country Name' to 'country_name_wb', and 'Indicator Code' to 'feature'.

wb_ts_df = wb_ts_df.rename({'Country Code': 'country_code',
                            'Country Name':'country_name_wb',
                            'Indicator Code':'feature'},
                            axis=1)
wb_ts_df

,country_code,country_name_wb,feature,1960,1961,1962,1963,1964,1965,1966,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ARB,Arab World,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85.644218,85.932567,86.232384,86.478597,86.722685,86.937933,87.040774,87.235539,87.307068,NaN
1,ARB,Arab World,EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,87.039588,88.992620,88.015356,88.681886,89.195062,90.324659,88.910749,89.999946,90.277735,NaN
2,ARB,Arab World,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.859225,11.638397,10.297891,6.237703,5.203895,6.480083,8.479070,7.437145,4.376374,NaN
3,ARB,Arab World,NY.ADJ.DFOR.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.032266,0.062625,0.084948,0.099784,0.095056,0.095498,0.051058,0.062195,0.079081,NaN
4,ARB,Arab World,AG.LND.AGRI.ZS,NaN,30.981194,30.982443,31.006834,31.017781,31.042245,31.05018,...,36.484107,36.532755,36.529508,36.565532,36.627752,36.673546,36.659112,36.597936,36.659647,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16008,ZWE,Zimbabwe,IC.LGL.CRED.XQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,6.000000,NaN,NaN
16009,ZWE,Zimbabwe,ER.PTD.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,27.214542,27.214585,27.214585,27.214746,27.214746,27.214746
16010,ZWE,Zimbabwe,SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.153000,4.982000,4.770000,4.778000,4.788000,4.785000,4.796000,4.833000,5.351000,5.174000
16011,ZWE,Zimbabwe,SP.UWT.TFRT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#Type noncountries.remove('World') to remove 'World' from the noncountries list. (We want to keep the total world data for now). 
#Then use the .query() method to remove the rows in which 'country_name_wb' is equal to one of the other entries in the noncountries list.

noncountries.remove('World')

In [19]:
wb_ts_df = wb_ts_df.query("country_name_wb != @noncountries")
wb_ts_df

,country_code,country_name_wb,feature,1960,1961,1962,1963,1964,1965,1966,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
2211,WLD,World,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,59.315789,60.417242,61.796860,63.068478,64.299537,65.773618,67.147032,68.457171,69.681839,NaN
2212,WLD,World,EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,84.740710,85.240416,85.648321,86.619520,87.696899,88.594052,89.566532,90.011785,90.521569,NaN
2213,WLD,World,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.999282,1.774908,1.515980,0.817549,0.714896,0.916155,1.239542,1.079196,0.791238,NaN
2214,WLD,World,NY.ADJ.DFOR.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.059747,0.061098,0.068719,0.070765,0.072687,0.069076,0.048353,0.049777,0.058882,NaN
2215,WLD,World,AG.LND.AGRI.ZS,NaN,34.55308,34.618111,34.702027,34.775809,34.871252,34.935279,...,36.629367,36.558281,36.550972,36.360611,36.336393,36.616665,36.498736,36.511307,36.458841,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16008,ZWE,Zimbabwe,IC.LGL.CRED.XQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,6.000000,NaN,NaN
16009,ZWE,Zimbabwe,ER.PTD.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,27.214542,27.214585,27.214585,27.214746,27.214746,27.214746
16010,ZWE,Zimbabwe,SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.153000,4.982000,4.770000,4.778000,4.788000,4.785000,4.796000,4.833000,5.351000,5.174000
16011,ZWE,Zimbabwe,SP.UWT.TFRT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
replace_map = {'AG.LND.AGRI.ZS': 'agricultural_land',
 'AG.LND.FRST.ZS': 'forest_area',
 'AG.PRD.FOOD.XD': 'food_production_index',
 'CC.EST': 'control_of_corruption',
 'EG.CFT.ACCS.ZS': 'access_to_clean_fuels_and_technologies_for_cooking',
 'EG.EGY.PRIM.PP.KD': 'energy_intensity_level_of_primary_energy',
 'EG.ELC.ACCS.ZS': 'access_to_electricity',
 'EG.ELC.COAL.ZS': 'electricity_production_from_coal_sources',
 'EG.ELC.RNEW.ZS': 'renewable_electricity_output',
 'EG.FEC.RNEW.ZS': 'renewable_energy_consumption',
 'EG.IMP.CONS.ZS': 'energy_imports',
 'EG.USE.COMM.FO.ZS': 'fossil_fuel_energy_consumption',
 'EG.USE.PCAP.KG.OE': 'energy_use',
 'EN.ATM.CO2E.PC': 'co2_emissions',
 'EN.ATM.METH.PC': 'methane_emissions',
 'EN.ATM.NOXE.PC': 'nitrous_oxide_emissions',
 'EN.ATM.PM25.MC.M3': 'pm2_5_air_pollution',
 'EN.CLC.CDDY.XD': 'cooling_degree_days',
 'EN.CLC.GHGR.MT.CE': 'ghg_net_emissions',
 'EN.CLC.HEAT.XD': 'heat_index_35',
 'EN.CLC.MDAT.ZS': 'droughts',
 'EN.CLC.PRCP.XD': 'maximum_5-day_rainfall',
 'EN.CLC.SPEI.XD': 'mean_drought_index',
 'EN.MAM.THRD.NO': 'mammal_species',
 'EN.POP.DNST': 'population_density',
 'ER.H2O.FWTL.ZS': 'annual_freshwater_withdrawals',
 'ER.PTD.TOTL.ZS': 'terrestrial_and_marine_protected_areas',
 'GB.XPD.RSDV.GD.ZS': 'research_and_development_expenditure',
 'GE.EST': 'government_effectiveness',
 'IC.BUS.EASE.XQ': 'ease_of_doing_business_rank',
 'IC.LGL.CRED.XQ': 'strength_of_legal_rights_index',
 'IP.JRN.ARTC.SC': 'scientific_and_technical_journal_articles',
 'IP.PAT.RESD': 'patent_applications',
 'IT.NET.USER.ZS': 'individuals_using_the_internet',
 'NV.AGR.TOTL.ZS': 'agriculture',
 'NY.ADJ.DFOR.GN.ZS': 'net_forest_depletion',
 'NY.ADJ.DRES.GN.ZS': 'natural_resources_depletion',
 'NY.GDP.MKTP.KD.ZG': 'gdp_growth',
 'PV.EST': 'political_stability_and_absence_of_violence',
 'RL.EST': 'rule_of_law',
 'RQ.EST': 'regulatory_quality',
 'SE.ADT.LITR.ZS': 'literacy_rate',
 'SE.ENR.PRSC.FM.ZS': 'gross_school_enrollment',
 'SE.PRM.ENRR': 'primary_school_enrollment',
 'SE.XPD.TOTL.GB.ZS': 'government_expenditure_on_education',
 'SG.GEN.PARL.ZS': 'proportion_of_seats_held_by_women_in_national_parliaments',
 'SH.DTH.COMM.ZS': 'cause_of_death',
 'SH.DYN.MORT': 'mortality_rate',
 'SH.H2O.SMDW.ZS': 'people_using_safely_managed_drinking_water_services',
 'SH.MED.BEDS.ZS': 'hospital_beds',
 'SH.STA.OWAD.ZS': 'prevalence_of_overweight',
 'SH.STA.SMSS.ZS': 'people_using_safely_managed_sanitation_services',
 'SI.DST.FRST.20': 'income_share_held_by_lowest_20pct',
 'SI.POV.GINI': 'gini_index',
 'SI.POV.NAHC': 'poverty_headcount_ratio_at_national_poverty_lines',
 'SI.SPR.PCAP.ZG': 'annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income',
 'SL.TLF.0714.ZS': 'children_in_employment',
 'SL.TLF.ACTI.ZS': 'labor_force_participation_rate',
 'SL.TLF.CACT.FM.ZS': 'ratio_of_female_to_male_labor_force_participation_rate',
 'SL.UEM.TOTL.ZS': 'unemployment',
 'SM.POP.NETM': 'net_migration',
 'SN.ITK.DEFC.ZS': 'prevalence_of_undernourishment',
 'SP.DYN.LE00.IN': 'life_expectancy_at_birth',
 'SP.DYN.TFRT.IN': 'fertility_rate',
 'SP.POP.65UP.TO.ZS': 'population_ages_65_and_above',
 'SP.UWT.TFRT': 'unmet_need_for_contraception',
 'VA.EST': 'voice_and_accountability'}

wb_ts_df.feature = wb_ts_df.feature.map(replace_map)
wb_ts_df

/tmp/ipykernel_114/2702446553.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wb_ts_df.feature = wb_ts_df.feature.map(replace_map)


,country_code,country_name_wb,feature,1960,1961,1962,1963,1964,1965,1966,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
2211,WLD,World,access_to_clean_fuels_and_technologies_for_coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,59.315789,60.417242,61.796860,63.068478,64.299537,65.773618,67.147032,68.457171,69.681839,NaN
2212,WLD,World,access_to_electricity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,84.740710,85.240416,85.648321,86.619520,87.696899,88.594052,89.566532,90.011785,90.521569,NaN
2213,WLD,World,natural_resources_depletion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.999282,1.774908,1.515980,0.817549,0.714896,0.916155,1.239542,1.079196,0.791238,NaN
2214,WLD,World,net_forest_depletion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.059747,0.061098,0.068719,0.070765,0.072687,0.069076,0.048353,0.049777,0.058882,NaN
2215,WLD,World,agricultural_land,NaN,34.55308,34.618111,34.702027,34.775809,34.871252,34.935279,...,36.629367,36.558281,36.550972,36.360611,36.336393,36.616665,36.498736,36.511307,36.458841,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16008,ZWE,Zimbabwe,strength_of_legal_rights_index,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,6.000000,NaN,NaN
16009,ZWE,Zimbabwe,terrestrial_and_marine_protected_areas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,27.214542,27.214585,27.214585,27.214746,27.214746,27.214746
16010,ZWE,Zimbabwe,unemployment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.153000,4.982000,4.770000,4.778000,4.788000,4.785000,4.796000,4.833000,5.351000,5.174000
16011,ZWE,Zimbabwe,unmet_need_for_contraception,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN


## f. 
The World Bank country-time-series dataset is strangely organized. The features are stored in the rows, when typically we would want these features to be columns. Also, years are stored in columns, when typically we would want years to be represented by different rows. We can repair this structure by reshaping the data. Following the discussion in chapter 9 of "Surfing the Data Pipeline with Python", perform the following reshapes:

 - First, turn the columns that refer to years into rows,

 - Then turn the rows that refer to features into columns. [3 points]



In [21]:
wb_ts_df = pd.melt(wb_ts_df, id_vars = ['country_code', 'country_name_wb', 'feature'], 
       value_vars = [str(i) for i in range(1960, 2022)], var_name = 'year')
wb_ts_df

,country_code,country_name_wb,feature,year,value
0,WLD,World,access_to_clean_fuels_and_technologies_for_coo...,1960,NaN
1,WLD,World,access_to_electricity,1960,NaN
2,WLD,World,natural_resources_depletion,1960,NaN
3,WLD,World,net_forest_depletion,1960,NaN
4,WLD,World,agricultural_land,1960,NaN
...,...,...,...,...,...
805871,ZWE,Zimbabwe,strength_of_legal_rights_index,2021,NaN
805872,ZWE,Zimbabwe,terrestrial_and_marine_protected_areas,2021,27.214746
805873,ZWE,Zimbabwe,unemployment,2021,5.174000
805874,ZWE,Zimbabwe,unmet_need_for_contraception,2021,NaN


In [22]:
wb_ts_df = wb_ts_df.pivot_table(index = ['country_code', 'country_name_wb', 'year'],
                                columns = 'feature',
                                values = 'value')
wb_ts_df

feature                            access_to_clean_fuels_and_technologies_for_cooking  \
country_code country_name_wb year                                                       
AFG          Afghanistan     1960                                                NaN    
                             1961                                                NaN    
                             1962                                                NaN    
                             1963                                                NaN    
                             1964                                                NaN    
...                                                                              ...    
ZWE          Zimbabwe        2017                                               29.8    
                             2018                                               29.9    
                             2019                                               30.1    
                             2020                                               30.4    
                             2021                                                NaN    

feature                            access_to_electricity  agricultural_land  \
country_code country_name_wb year                                             
AFG          Afghanistan     1960                    NaN                NaN   
                             1961                    NaN          57.801696   
                             1962                    NaN          57.893688   
                             1963                    NaN          57.970348   
                             1964                    NaN          58.066940   
...                                                  ...                ...   
ZWE          Zimbabwe        2017              44.178635          41.876696   
                             2018              45.572647          41.876696   
                             2019              46.781475          41.876696   
                             2020              52.747669          41.876696   
                             2021                    NaN                NaN   

feature                            agriculture  annual_freshwater_withdrawals  \
country_code country_name_wb year                                               
AFG          Afghanistan     1960          NaN                            NaN   
                             1961          NaN                            NaN   
                             1962          NaN                            NaN   
                             1963          NaN                            NaN   
                             1964          NaN                            NaN   
...                                        ...                            ...   
ZWE          Zimbabwe        2017     8.340969                      27.234910   
                             2018    10.520955                      30.761664   
                             2019    10.143657                            NaN   
                             2020     7.607101                            NaN   
                             2021          NaN                            NaN   

feature                            annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income  \
country_code country_name_wb year                                                                                        
AFG          Afghanistan     1960                                                NaN                                     
                             1961                                                NaN                                     
                             1962                                                NaN                                     
                             1963                                                NaN                                     
                            

In [23]:
wb_ts_df = pd.DataFrame(wb_ts_df.to_records())
wb_ts_df

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income,cause_of_death,...,renewable_electricity_output,renewable_energy_consumption,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,unemployment,unmet_need_for_contraception,voice_and_accountability
0,AFG,Afghanistan,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1961,NaN,NaN,57.801696,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1962,NaN,NaN,57.893688,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1963,NaN,NaN,57.970348,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1964,NaN,NaN,58.066940,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12017,ZWE,Zimbabwe,2017,29.8,44.178635,41.876696,8.340969,27.234910,-3.5,NaN,...,NaN,82.46,NaN,-1.385256,340.45,5.0,27.214585,4.785,NaN,-1.195904
12018,ZWE,Zimbabwe,2018,29.9,45.572647,41.876696,10.520955,30.761664,NaN,NaN,...,NaN,80.23,NaN,-1.278995,359.33,5.0,27.214585,4.796,NaN,-1.140975
12019,ZWE,Zimbabwe,2019,30.1,46.781475,41.876696,10.143657,NaN,NaN,47.647301,...,NaN,81.50,NaN,-1.292071,NaN,6.0,27.214746,4.833,NaN,-1.164705
12020,ZWE,Zimbabwe,2020,30.4,52.747669,41.876696,7.607101,NaN,NaN,NaN,...,NaN,NaN,NaN,-1.307897,NaN,NaN,27.214746,5.351,NaN,-1.113716


## g.
After these reshapes, the year column in the World Bank country-time-series dataset is stored as a string. Convert this column to an integer data type. [1 point]

In [24]:
wb_ts_df['year'] = wb_ts_df['year'].astype(int)
wb_ts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12022 entries, 0 to 12021
Data columns (total 66 columns):
 #   Column                                                                               Non-Null Count  Dtype  
---  ------                                                                               --------------  -----  
 0   country_code                                                                         12022 non-null  object 
 1   country_name_wb                                                                      12022 non-null  object 
 2   year                                                                                 12022 non-null  int64  
 3   access_to_clean_fuels_and_technologies_for_cooking                                   4007 non-null   float64
 4   access_to_electricity                                                                5226 non-null   float64
 5   agricultural_land                                                                    114

## h. 
Create a new dataframe containing just the rows from the World Bank country-time-series dataset that refer to the whole world. Then remove the "World" rows from the World Bank country-time-series dataset. Drop the 'country_code' and 'country_name_wb' from the world dataset. Finally, with the exception of the year column, rename all columns in the world dataset by adding "world_" in front of the column name. [2 points]

In [25]:
world_ts = wb_ts_df.copy()
world_ts = world_ts.query("country_name_wb == 'World'")
world_ts

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income,cause_of_death,...,renewable_electricity_output,renewable_energy_consumption,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,unemployment,unmet_need_for_contraception,voice_and_accountability
11650,WLD,World,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11651,WLD,World,1961,NaN,NaN,34.553080,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11652,WLD,World,1962,NaN,NaN,34.618111,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11653,WLD,World,1963,NaN,NaN,34.702027,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11654,WLD,World,1964,NaN,NaN,34.775809,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11707,WLD,World,2017,65.773618,88.594052,36.616665,4.139864,9.241538,NaN,NaN,...,NaN,NaN,2.133745,NaN,2.464402e+06,5.347368,14.339432,5.557650,NaN,NaN
11708,WLD,World,2018,67.147032,89.566532,36.498736,3.941813,9.250457,NaN,NaN,...,NaN,NaN,2.196333,NaN,2.554319e+06,5.507853,14.339452,5.389680,NaN,NaN
11709,WLD,World,2019,68.457171,90.011785,36.511307,4.007295,NaN,NaN,18.40885,...,NaN,NaN,2.331913,NaN,NaN,5.685864,NaN,5.356915,NaN,NaN
11710,WLD,World,2020,69.681839,90.521569,36.458841,4.385748,NaN,NaN,NaN,...,NaN,NaN,2.625844,NaN,NaN,NaN,NaN,6.573234,NaN,NaN


In [26]:
wb_ts_df = wb_ts_df.query("country_name_wb != 'World'")
wb_ts_df

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income,cause_of_death,...,renewable_electricity_output,renewable_energy_consumption,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,unemployment,unmet_need_for_contraception,voice_and_accountability
0,AFG,Afghanistan,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1961,NaN,NaN,57.801696,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1962,NaN,NaN,57.893688,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1963,NaN,NaN,57.970348,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1964,NaN,NaN,58.066940,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12017,ZWE,Zimbabwe,2017,29.8,44.178635,41.876696,8.340969,27.234910,-3.5,NaN,...,NaN,82.46,NaN,-1.385256,340.45,5.0,27.214585,4.785,NaN,-1.195904
12018,ZWE,Zimbabwe,2018,29.9,45.572647,41.876696,10.520955,30.761664,NaN,NaN,...,NaN,80.23,NaN,-1.278995,359.33,5.0,27.214585,4.796,NaN,-1.140975
12019,ZWE,Zimbabwe,2019,30.1,46.781475,41.876696,10.143657,NaN,NaN,47.647301,...,NaN,81.50,NaN,-1.292071,NaN,6.0,27.214746,4.833,NaN,-1.164705
12020,ZWE,Zimbabwe,2020,30.4,52.747669,41.876696,7.607101,NaN,NaN,NaN,...,NaN,NaN,NaN,-1.307897,NaN,NaN,27.214746,5.351,NaN,-1.113716


In [27]:
world_ts = world_ts.drop(['country_code', 'country_name_wb'], axis = 1)
world_ts.columns = ['year'] + ['world_' + str(col) for col in world_ts.columns if col != 'year']
world_ts

,year,world_access_to_clean_fuels_and_technologies_for_cooking,world_access_to_electricity,world_agricultural_land,world_agriculture,world_annual_freshwater_withdrawals,world_annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income,world_cause_of_death,world_children_in_employment,world_co2_emissions,...,world_renewable_electricity_output,world_renewable_energy_consumption,world_research_and_development_expenditure,world_rule_of_law,world_scientific_and_technical_journal_articles,world_strength_of_legal_rights_index,world_terrestrial_and_marine_protected_areas,world_unemployment,world_unmet_need_for_contraception,world_voice_and_accountability
11650,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11651,1961,NaN,NaN,34.553080,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11652,1962,NaN,NaN,34.618111,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11653,1963,NaN,NaN,34.702027,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11654,1964,NaN,NaN,34.775809,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11707,2017,65.773618,88.594052,36.616665,4.139864,9.241538,NaN,NaN,NaN,4.457093,...,NaN,NaN,2.133745,NaN,2.464402e+06,5.347368,14.339432,5.557650,NaN,NaN
11708,2018,67.147032,89.566532,36.498736,3.941813,9.250457,NaN,NaN,NaN,4.510145,...,NaN,NaN,2.196333,NaN,2.554319e+06,5.507853,14.339452,5.389680,NaN,NaN
11709,2019,68.457171,90.011785,36.511307,4.007295,NaN,NaN,18.40885,NaN,4.469660,...,NaN,NaN,2.331913,NaN,NaN,5.685864,NaN,5.356915,NaN,NaN
11710,2020,69.681839,90.521569,36.458841,4.385748,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.625844,NaN,NaN,NaN,NaN,6.573234,NaN,NaN


## i. 
Next we will merge the World Bank country-time-series dataset with the V-Dem dataset, matching on the 'country_code' and 'year' columns. First, write a sentence stating whether you expect this merge to be one-to-one, many-to-one, one-to-many, or many-to-many, and describe your rationale. Second, merge the two datasets together in a way that checks whether your expectation is met, and also allows you to see the rows that failed to match. [3 points]


Ans: I expect the merge to be one-to-one, as v-dem is also time series and both contain country code and the years 1960-2021.

In [28]:
v_dem_df.query("country_code == 'MEX'")

,country_code,country_name_vdem,year,democracy
171,MEX,Mexico,1960,0.231
172,MEX,Mexico,1961,0.232
173,MEX,Mexico,1962,0.234
174,MEX,Mexico,1963,0.234
175,MEX,Mexico,1964,0.234
...,...,...,...,...
228,MEX,Mexico,2017,0.630
229,MEX,Mexico,2018,0.674
230,MEX,Mexico,2019,0.669
231,MEX,Mexico,2020,0.647


In [29]:
wb_ts_df.query("country_code == 'MEX'")

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income,cause_of_death,...,renewable_electricity_output,renewable_energy_consumption,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,unemployment,unmet_need_for_contraception,voice_and_accountability
6879,MEX,Mexico,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6880,MEX,Mexico,1961,NaN,NaN,50.575375,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6881,MEX,Mexico,1962,NaN,NaN,50.504900,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6882,MEX,Mexico,1963,NaN,NaN,50.468891,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6883,MEX,Mexico,1964,NaN,NaN,50.438026,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,MEX,Mexico,2017,84.8,100.000000,49.870624,3.381074,21.476773,NaN,NaN,...,NaN,9.99,0.32832,-0.569109,16004.90,10.0,19.056240,3.420,NaN,-0.077817
6937,MEX,Mexico,2018,84.8,99.500000,49.649425,3.383800,21.721271,NaN,NaN,...,NaN,10.03,0.30711,-0.626365,16345.64,10.0,19.056240,3.280,10.8,-0.016886
6938,MEX,Mexico,2019,84.9,99.599998,49.438514,3.394313,NaN,NaN,9.093437,...,NaN,10.34,0.28384,-0.646011,NaN,10.0,18.909851,3.480,NaN,0.007819
6939,MEX,Mexico,2020,84.9,99.400002,49.969392,3.798496,NaN,-0.68,NaN,...,NaN,NaN,0.30096,-0.647769,NaN,NaN,18.909849,4.450,NaN,-0.025030


In [30]:
v_dem_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10371 entries, 171 to 25980
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   country_code       10371 non-null  object 
 1   country_name_vdem  10371 non-null  object 
 2   year               10371 non-null  int64  
 3   democracy          10369 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 405.1+ KB


In [31]:
country_ts = pd.merge(v_dem_df, wb_ts_df,
                        on = ['country_code', 'year'],
                        how = 'outer',
                        validate = 'one_to_one',
                        indicator = 'matched')

## j.
After this merge, use the .value_counts() method to see the total numnber of observations that were found in both datasets, the number found only in the left dataset, and the number found only in the right dataset. (If you entered the World Bank dataset into the merge function first, then "left_only" refers to the rows found in the World Bank but not V-Dem, and "right_only refers to the rows found in V-Dem but not the World Bank.) There should be more than 9000 rows that matched, but more than 2000 that failed to match.

Then conduct two data aggregations to help us investigate why these observations did not match:

- First use .query() to keep only the observations that were present in the World Bank data but not V-Dem. (These are the 'left_only' observations if you typed the World Bank data into the merge function first.) Use .groupby() to aggregate the data by 'country_code' and 'country_name_wb'. Then save the minimum and maximum values of 'year' for each country.

- Then use .query() to keep only the observations that were present in the V-Dem data but not the World Bank. Use .groupby() to aggregate the data by 'country_code' and 'country_name_vdem'. Then save the minimum and maximum values of 'year' for each country. [3 points]

In [32]:
country_ts['matched'].value_counts()

both          9976
right_only    1984
left_only      395
Name: matched, dtype: int64

In [33]:
wb_mins = country_ts.query("matched == 'right_only'")[['year', 'country_code', 'country_name_wb']].groupby(['country_code', 'country_name_wb']).agg({'year': 'min'})
wb_mins

,,year
country_code,country_name_wb,
AND,Andorra,1961
ARE,United Arab Emirates,1960
ARM,Armenia,1960
ATG,Antigua and Barbuda,1960
AZE,Azerbaijan,1960
BGD,Bangladesh,1960
BHS,"Bahamas, The",1960
BIH,Bosnia and Herzegovina,1960
BLR,Belarus,1960


In [34]:
wb_maxes = country_ts.query("matched == 'right_only'")[['year', 'country_code', 'country_name_wb']].groupby(['country_code', 'country_name_wb']).agg({'year': 'max'})
wb_maxes

,,year
country_code,country_name_wb,
AND,Andorra,2021
ARE,United Arab Emirates,1970
ARM,Armenia,1989
ATG,Antigua and Barbuda,2021
AZE,Azerbaijan,1989
BGD,Bangladesh,1970
BHS,"Bahamas, The",2021
BIH,Bosnia and Herzegovina,1991
BLR,Belarus,1989


In [35]:
v_dem_mins = country_ts.query("matched == 'left_only'")[['year', 'country_code', 'country_name_vdem']].groupby(['country_code', 'country_name_vdem']).agg({'year': 'min'})
v_dem_mins

,,year
country_code,country_name_vdem,
DDR,German Democratic Republic,1960
HKG,Hong Kong,1960
PSE,Palestine/West Bank,1967
PSG,Palestine/Gaza,1960
SML,Somaliland,1991
TWN,Taiwan,1960
VDR,Republic of Vietnam,1960
XKX,Kosovo,1999
YMD,South Yemen,1960


In [36]:
v_dem_maxes = country_ts.query("matched == 'left_only'")[['year', 'country_code', 'country_name_vdem']].groupby(['country_code', 'country_name_vdem']).agg({'year': 'max'})
v_dem_maxes

,,year
country_code,country_name_vdem,
DDR,German Democratic Republic,1990
HKG,Hong Kong,2021
PSE,Palestine/West Bank,2021
PSG,Palestine/Gaza,2021
SML,Somaliland,2021
TWN,Taiwan,2021
VDR,Republic of Vietnam,1975
XKX,Kosovo,2021
YMD,South Yemen,1990


In [37]:
wb_minmax= pd.merge(wb_mins, wb_maxes,
                        on = ['country_code', 'country_name_wb'],
                        how = 'outer')
wb_minmax

,,year_x,year_y
country_code,country_name_wb,,
AND,Andorra,1961,2021
ARE,United Arab Emirates,1960,1970
ARM,Armenia,1960,1989
ATG,Antigua and Barbuda,1960,2021
AZE,Azerbaijan,1960,1989
BGD,Bangladesh,1960,1970
BHS,"Bahamas, The",1960,2021
BIH,Bosnia and Herzegovina,1960,1991
BLR,Belarus,1960,1989


In [38]:
vdem_minmax= pd.merge(v_dem_mins, v_dem_maxes,
                        on = ['country_code', 'country_name_vdem'],
                        how = 'outer')
vdem_minmax

,,year_x,year_y
country_code,country_name_vdem,,
DDR,German Democratic Republic,1960,1990
HKG,Hong Kong,1960,2021
PSE,Palestine/West Bank,1967,2021
PSG,Palestine/Gaza,1960,2021
SML,Somaliland,1991,2021
TWN,Taiwan,1960,2021
VDR,Republic of Vietnam,1960,1975
XKX,Kosovo,1999,2021
YMD,South Yemen,1960,1990


k. Here's where a deep understanding of the data becomes very important. There are two reasons why an observation may fail to match in a merge. One reason is a difference in spelling. Suppose that South Korea (which is also known as the Republic of Korea) is coded as SKO in the World Bank data and ROK in V-Dem. In this case, we should recode one or the other of SKO and ROK so that they match, otherwise we will lose the data on South Korea. But the second reason why observations might fail to match is due to differences in coverage in the data collection strategy: it is possible that a country wasn't included in one data's coverage, or that certain years for that country were not included. For differences in coverage, there's no way to manipulate the data to match, so we are out of luck and we have to either delete these observations or proceed with missing data from one of the data sources.

Take a close look at the two data aggregation tables you generated in part (j), and answer the following questions:

- Do you see any countries that are present in both the unmatched World Bank rows and the unmatched V-Dem rows, but with different spellings?

- Do some digging on Wikipedia and other sources on the Internet. What do you think is the primary reason why some countries are present in the V-Dem data but not the World Bank? (You don't need to describe the reasoning for every country. Just dig until you see a general pattern and describe it here.)

- Do some more digging on Wikipedia and other sources on the Internet. What do you think is the primary reason why some countries are present in the World Bank data but not V-Dem? (You don't need to describe the reasoning for every country. Just dig until you see a general pattern and describe it here.) [3 points]



**Ans**

- I don't see any countries with mismatched spelling.

- V-Dem seems to include states that are not nationally recognized or have ceased to exist. For example, Somaliland is considered nationally to be part of Somalia and Kosovo is only a partially recognized state. This is probably why the World Bank does not collect data on these countries, but V-Dem has decided to. 

- Many of the countries in the World Bank dataset, but not in the V-Dem dataset, are former Soviet states, smaller countries/microstates like Monaco or Andorra, or countries that came into existence between 1960 and 2021 like South Sudan or St. Vincent and the Grenadines. 

## l. 
Once you are convinced that all of the unmatched observations are due to differences in the coverage of the data collection strategies of the World Bank and V-Dem, repeat the merge, this time dropping all unmatched observations. This time there is no need to validate the type of merge, and no need to define a variable to indicate matching. [1 point]

In [39]:
country_ts = pd.merge(v_dem_df, wb_ts_df,
                        on = ['country_code', 'year'],
                        how = 'inner')
country_ts

,country_code,country_name_vdem,year,democracy,country_name_wb,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,...,renewable_electricity_output,renewable_energy_consumption,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,unemployment,unmet_need_for_contraception,voice_and_accountability
0,MEX,Mexico,1960,0.231,Mexico,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MEX,Mexico,1961,0.232,Mexico,NaN,NaN,50.575375,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MEX,Mexico,1962,0.234,Mexico,NaN,NaN,50.504900,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MEX,Mexico,1963,0.234,Mexico,NaN,NaN,50.468891,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MEX,Mexico,1964,0.234,Mexico,NaN,NaN,50.438026,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,HUN,Hungary,2017,0.560,Hungary,100.0,100.0,58.108700,3.753567,74.983333,...,NaN,14.54,1.31680,0.559209,6645.69,9.0,22.597558,4.160,NaN,0.536953
9972,HUN,Hungary,2018,0.483,Hungary,100.0,100.0,58.031996,3.485912,73.833333,...,NaN,13.58,1.50755,0.578446,6700.92,9.0,22.597558,3.710,NaN,0.478424
9973,HUN,Hungary,2019,0.460,Hungary,100.0,100.0,57.834758,3.342482,NaN,...,NaN,13.64,1.47736,0.521551,NaN,9.0,22.598783,3.420,NaN,0.343969
9974,HUN,Hungary,2020,0.462,Hungary,100.0,100.0,53.725619,3.380822,NaN,...,NaN,NaN,1.60766,0.508961,NaN,NaN,22.597372,4.250,NaN,0.391829


# (3) 
After working through all of the data wrangling steps in problem 2, you now have the following data frames:

The World Bank's country level indicators (we'll call this table "country")

The World Bank's indicators for the whole world through the 1960-2021 timeframe (we'll call this table "world")

The merged data frame containing both the World Bank's and V-Dem's country-time-series data (we'll call this table "timeseries")

Do these three tables together comprise a third normal form database? Write a paragraph that describes your reasoning, and be clear about whether the criteria or 1st, 2nd, and 3rd normal forms have been achieved. If the data are not in 3rd normal form, take steps to put the data in 3rd normal form. (Hint: it's not important to keep the country names from both the World Bank and V-Dem. If one of these names is stored in one of the tables, that works.) [4 points]

In [40]:
country = esg_country_df
country.columns

Index(['country_code', 'country_name_wb', 'country_longname', 'currency_unit',
       'region', 'income_group'],
      dtype='object')

In [41]:
world = world_ts
world.columns

Index(['year', 'world_access_to_clean_fuels_and_technologies_for_cooking',
       'world_access_to_electricity', 'world_agricultural_land',
       'world_agriculture', 'world_annual_freshwater_withdrawals',
       'world_annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income',
       'world_cause_of_death', 'world_children_in_employment',
       'world_co2_emissions', 'world_control_of_corruption', 'world_droughts',
       'world_ease_of_doing_business_rank',
       'world_electricity_production_from_coal_sources',
       'world_energy_imports',
       'world_energy_intensity_level_of_primary_energy', 'world_energy_use',
       'world_fertility_rate', 'world_food_production_index',
       'world_forest_area', 'world_fossil_fuel_energy_consumption',
       'world_gdp_growth', 'world_ghg_net_emissions', 'world_gini_index',
       'world_government_effectiveness',
       'world_government_expenditure_on_education',
       'world_gross_school_enrollment', 'worl

In [42]:
timeseries = country_ts
timeseries.columns

Index(['country_code', 'country_name_vdem', 'year', 'democracy',
       'country_name_wb', 'access_to_clean_fuels_and_technologies_for_cooking',
       'access_to_electricity', 'agricultural_land', 'agriculture',
       'annual_freshwater_withdrawals',
       'annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income',
       'cause_of_death', 'children_in_employment', 'co2_emissions',
       'control_of_corruption', 'droughts', 'ease_of_doing_business_rank',
       'electricity_production_from_coal_sources', 'energy_imports',
       'energy_intensity_level_of_primary_energy', 'energy_use',
       'fertility_rate', 'food_production_index', 'forest_area',
       'fossil_fuel_energy_consumption', 'gdp_growth', 'ghg_net_emissions',
       'gini_index', 'government_effectiveness',
       'government_expenditure_on_education', 'gross_school_enrollment',
       'hospital_beds', 'income_share_held_by_lowest_20pct',
       'individuals_using_the_internet', 'labor_forc

1st normal form would indicate that each table has a primary key and there is no non-atomic data. For our country table, we have a primary key, which is country code and there is no non-atomic data. For our world table, our primary key is year, and there is no non-atomic data. For our timeseries data, country code and year is our primary/superkey and there is no non-atomic data. All of the requirements have been filled for 1st normal form, so there we have it! Our data is in 1st normal form. Next, for our data to be in 2nd normal form, all our tables must be in 1st normal form, which we just checked. The other condition to be met is that every non-prime column must depend on the entire primary key and not just part of the primary key. The only table we would need to check is our timeseries table since it has a superkey. Each column varies by year and country, except 'country_name_vdem' and 'country_name_wb', both of which depend on 'country_code'. We already have 'country_name_wb' in our country table and 'country_name_vdem' is redundant, so let's drop these from our table. 

In [43]:
timeseries = timeseries.drop(['country_name_vdem', 'country_name_wb'], axis = 1)
timeseries.columns

Index(['country_code', 'year', 'democracy',
       'access_to_clean_fuels_and_technologies_for_cooking',
       'access_to_electricity', 'agricultural_land', 'agriculture',
       'annual_freshwater_withdrawals',
       'annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income',
       'cause_of_death', 'children_in_employment', 'co2_emissions',
       'control_of_corruption', 'droughts', 'ease_of_doing_business_rank',
       'electricity_production_from_coal_sources', 'energy_imports',
       'energy_intensity_level_of_primary_energy', 'energy_use',
       'fertility_rate', 'food_production_index', 'forest_area',
       'fossil_fuel_energy_consumption', 'gdp_growth', 'ghg_net_emissions',
       'gini_index', 'government_effectiveness',
       'government_expenditure_on_education', 'gross_school_enrollment',
       'hospital_beds', 'income_share_held_by_lowest_20pct',
       'individuals_using_the_internet', 'labor_force_participation_rate',
       'life_expe

Now our timeseries table is in 2nd normal form and so are the rest of our tables. Our data is in 2nd normal form. For our data to be in 3rd normal form, we have to satisfy the 2nd normal form, which we have, and that there are no transitive dependencies, or a functional dependence between two non-prime attributes, unless the funtional dependence is based on a *candidate key*. We do have transitive dependencies in our country table. However, 'country_name_wb' or 'country_longname' could serve as the primary key, so they are candidate keys and are allowed to have functional dependencies. There are no other functional dependencies in our data, so we are a go on 3rd normal form.

# (4)
Connect to the PostgreSQL server that is running on port 5432. Create a new database called 'cardib'. Then use sqlalchemy to create an engine that connects to this database, and upload the country, world, and timeseries tables to this database. (Hint: set 'user' in the psycopg2.connect() method to be 'postgres', and set 'host' to be whatever you named the PostgreSQL service in your compose.yaml file. I named this service 'postgres' in my file. In addition, change 'localhost' in the create_engine() function to the name of this service as well.) [4 points]

In [44]:
postgres_password = os.environ['POSTGRES_PASSWORD']

In [45]:
server = psycopg2.connect(
    user = 'postgres',
    password = postgres_password,
    host = 'postgres',
    port = '5432')
server.autocommit = True

In [46]:
cursor = server.cursor()

In [47]:
try:
    cursor.execute("CREATE DATABASE cardib")
except:
    cursor.execute("DROP DATABASE cardib")
    cursor.execute("CREATE DATABASE cardib")

In [48]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = postgres_password,
    host = 'postgres',
    port = '5432',
    db = 'cardib'))

In [49]:
country.to_sql('country', con = engine, if_exists = 'replace', index=False, chunksize = 1000)
world.to_sql('world', con = engine, if_exists = 'replace', index=False, chunksize = 1000)
timeseries.to_sql('timeseries', con = engine, if_exists = 'replace', index=False, chunksize = 1000)

9976

# (5)
Use dbdocs to create an ER diagram for this database. Copy the text of your .dmbl file and paste it in a "raw" type cell below. Then copy the URL of the published ER diagram and paste it in a markdown cell.

A few points to keep in mind:

- In the timeseries table, country_code is varchar, year is int, and every feature is float.

- Pay close attention to whether the tables have a one-to-one, many-to-one, or one-to-many relationship with each other, and make sure the ER diagram properly represents that relationship.

- It would be good to write a sentence or two describing the sources of the data as a note for the database. [5 points]

https://dbdocs.io/beauleb1/Country_Analysis_Relational_DataBase?view=relationships

# (6) 
Write SQL queries for the cardib database that perform the following tasks, and display the output: [3 points each]

## a.
What countries had the highest quality democracies in the year 2021? Join the timeseries and country tables, select the country_name_wb and democracy attributes, rename county_name_wb to country, filter the rows to only those for the year 2021, and sort the rows in descending order by the value of democracy.

In [101]:
pd.set_option('display.max_rows', None)

In [102]:
myquery = '''
SELECT c.country_name_wb AS country, t.democracy 
FROM timeseries t
INNER JOIN country c
    ON t.country_code = c.country_code
WHERE t.year = 2021
ORDER BY t.democracy DESC
'''

democracy = pd.read_sql(myquery, con=engine)
democracy

,country,democracy
0,Denmark,0.908
1,Sweden,0.907
2,Costa Rica,0.900
3,Norway,0.898
4,Estonia,0.891
5,New Zealand,0.887
6,Ireland,0.885
7,Switzerland,0.885
8,Belgium,0.885
9,Portugal,0.883


## b.
How does the life expectancy at birth for Chile compare to the global average life expectancy at birth over the 1960-2021 time span? Join the timeseries and world tables, select year from the timeseries table and the life expectancy attributes from both the timeseries and world tables, filter the rows to just those where the country code is CHL, and sort the rows by year.

In [104]:
myquery = '''
SELECT t.year, t.life_expectancy_at_birth, w.world_life_expectancy_at_birth
FROM timeseries t
INNER JOIN world w
    ON t.year = w.year
WHERE t.country_code = 'CHL'
ORDER BY t.year DESC
'''

life_expectancy = pd.read_sql(myquery, con=engine)
life_expectancy

,year,life_expectancy_at_birth,world_life_expectancy_at_birth
0,2021,NaN,NaN
1,2020,80.329,72.747919
2,2019,80.181,72.763223
3,2018,80.042,72.574415
4,2017,79.909,72.391699
5,2016,79.779,72.186230
6,2015,79.646,71.952115
7,2014,79.504,71.746055
8,2013,79.349,71.465864
9,2012,79.176,71.173299


## c.
What regions of the world generated the most carbon dioxide emissions in 2019? Join the timeseries and country tables, filter the rows to only the ones that refer to 2019, and aggregate the data by region. Keep the region column from the country table and the sum of the co2_emissions attribute from the timeseries table. Rename the sum to co2_emissions. Then sort the rows in descending order by co2_emissions.

In [62]:
myquery = '''
SELECT c.region, SUM(t.co2_emissions) as co2_emissions
FROM timeseries t
INNER JOIN country c
    ON t.country_code = c.country_code
WHERE t.year = 2019
GROUP BY c.region
ORDER BY co2_emissions DESC
'''

co2 = pd.read_sql(myquery, con=engine)
co2

,region,co2_emissions
0,Europe & Central Asia,275.817873
1,Middle East & North Africa,175.104550
2,East Asia & Pacific,95.228469
3,Latin America & Caribbean,70.512349
4,Sub-Saharan Africa,43.942360
5,North America,30.104023
6,South Asia,10.304685


## d. 
What countries experienced the greatest increases in democratic quality over the 1960-2021 time span? Use subqueries to create two new tables on the fly. One table contains country_code and democracy from the timeseries table in which only the rows from 1960 are present. The second table contains country_code and democracy from the timeseries table in which only the rows from 2021 are present. Join these two tables together, then join the result with the country table. Select the country name from county, democracy from the 1960 table, democracy from the 2021 table, and the difference between the two democracy scores. Give each column a more intuitive name. Then sort the rows in descending order by the democracy difference and display the first 10 rows.

In [76]:
myquery = '''
WITH start_democracy AS (SELECT country_code, democracy 
FROM timeseries
WHERE year = 1960),

end_democracy AS (SELECT country_code, democracy 
FROM timeseries
WHERE year = 2021),

composite_democracy AS (SELECT s.country_code, s.democracy as democracy_1960, e.democracy as democracy_2021
FROM start_democracy s
INNER JOIN end_democracy e
    ON s.country_code = e.country_code)
    
SELECT c.country_name_wb as country,
       com.democracy_1960, 
       com.democracy_2021, 
       (CAST(com.democracy_2021 AS FLOAT) - CAST(com.democracy_1960 AS FLOAT)) AS democracy_improvement
FROM composite_democracy com
INNER JOIN country c
    ON c.country_code = com.country_code
ORDER BY democracy_improvement DESC
LIMIT 10
'''

democracy = pd.read_sql(myquery, con=engine)
democracy

,country,democracy_1960,democracy_2021,democracy_improvement
0,Spain,0.070,0.856,0.786
1,Portugal,0.128,0.883,0.755
2,Cabo Verde,0.025,0.747,0.722
3,Vanuatu,0.080,0.777,0.697
4,Timor-Leste,0.017,0.680,0.663
5,São Tomé and Principe,0.030,0.688,0.658
6,Romania,0.157,0.782,0.625
7,Czech Republic,0.194,0.805,0.611
8,Malawi,0.041,0.618,0.577
9,Namibia,0.061,0.634,0.573


## e. 
By count of countries, what is the most commonly used currency in the world?

In [79]:
myquery = '''
SELECT currency_unit, COUNT(currency_unit) as total_countries
FROM country
GROUP BY currency_unit
ORDER BY total_countries DESC
LIMIT 1
'''

currency = pd.read_sql(myquery, con=engine)
currency

,currency_unit,total_countries
0,Euro,23


## f. 
The GINI index measures the amount of economic inequality in a country. The higher the index, the greater the economic disparity between rich and poor. How does the average GINI index compare across income groups (see the country table) in 2019?

In [93]:
myquery = '''
SELECT c.income_group, AVG(t.gini_index) as avg_gini
FROM country c
INNER JOIN timeseries t
    ON t.country_code = c.country_code
WHERE year = 2019
GROUP BY income_group
ORDER BY avg_gini DESC
'''

gini = pd.read_sql(myquery, con=engine)
gini

,income_group,avg_gini
0,None,NaN
1,Low income,40.600000
2,Upper middle income,38.390476
3,Lower middle income,38.170000
4,High income,31.752000


## Extra credit
Some countries include the word 'Republic' or 'Democratic' in their official names. Other countries do not. Which of these two groups of countries had a higher average level of democratic quality in 2021? (It's OK if you use two queries to provide this answer.) [5 bonus points]

In [105]:
myquery = '''
SELECT AVG(t.democracy) AS avg_democracy, 
       CASE 
           WHEN c.country_longname LIKE '%%Rep%%' OR c.country_longname LIKE '%%Dem%%' THEN 1
           ELSE 0 
        END as rep_or_dem
FROM country c
INNER JOIN timeseries t
    ON t.country_code = c.country_code
WHERE year = 2021
GROUP BY rep_or_dem
'''

repdem = pd.read_sql(myquery, con=engine)
repdem

,avg_democracy,rep_or_dem
0,0.542902,0
1,0.497000,1


In [108]:
myquery = '''
SELECT c.country_code, t.democracy,
       CASE 
           WHEN c.country_longname LIKE '%%Rep%%' OR c.country_longname LIKE '%%Dem%%' THEN 1
           ELSE 0 
        END as rep_or_dem
FROM country c
INNER JOIN timeseries t
    ON t.country_code = c.country_code
WHERE year = 2021
'''

repdem = pd.read_sql(myquery, con=engine)
repdem

,country_code,democracy,rep_or_dem
0,MEX,0.628,0
1,SUR,0.760,1
2,SWE,0.907,0
3,CHE,0.885,0
4,GHA,0.663,1
5,ZAF,0.715,1
6,JPN,0.835,0
7,MMR,0.126,1
8,RUS,0.257,0
9,ALB,0.478,1
